In [1]:
import blpapi
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

A installer sur les ordi bloom : 
python -m pip install --index-url=https://bcms.bloomberg.com/pip/simple blpapi

In [2]:
# Bloomberg names : 
DATE = blpapi.Name("date")
ERROR_INFO = blpapi.Name("errorInfo")
EVENT_TIME = blpapi.Name("EVENT_TIME")
FIELD_DATA = blpapi.Name("fieldData")
FIELD_EXCEPTIONS = blpapi.Name("fieldExceptions")
FIELD_ID = blpapi.Name("fieldId")
SECURITY = blpapi.Name("security")
SECURITY_DATA = blpapi.Name("securityData")

In [3]:
class BLP():    
    
    def __init__(self):

        # Create Session object
        self.session = blpapi.Session()
        
        # Exit if can't start the Session
        if not self.session.start():
            print("Failed to start session.")
            return
        
        # Open & Get RefData Service or exit if impossible
        if not self.session.openService("//blp/refdata"):
            print("Failed to open //blp/refdata")
            return
        
        self.session.openService('//BLP/refdata')
        self.refDataSvc = self.session.getService('//BLP/refdata')

        print('Session open')
    
    #-----------------------------------------------------------------------------------------------------
    #------------------------------------------ BDH ------------------------------------------------------
    #-----------------------------------------------------------------------------------------------------
    
    def bdh(self, strSecurity, strFields, startdate, enddate, per='DAILY', perAdj = 'CALENDAR', 
            days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = "EUR"):
        
        # def bdh(self, strSecurity, strFields, startdate, enddate, per='DAILY', perAdj = 'CALENDAR', 
        #    days = 'NON_TRADING_WEEKDAYS', fill = 'PREVIOUS_VALUE', curr = None)
        
        """
            Summary:
                HistoricalDataRequest ; 
                Gets historical data for a set of securities and fields

            Inputs:
                strSecurity: list of str : list of tickers
                strFields: list of str : list of fields, must be static fields (e.g. px_last instead of last_price)
                startdate & enddate: dates
                per: periodicitySelection; daily, monthly, quarterly, semiannually or annually
                perAdj: periodicityAdjustment: ACTUAL, CALENDAR, FISCAL
                curr: string, else default currency is used 
                Days: nonTradingDayFillOption : NON_TRADING_WEEKDAYS*, ALL_CALENDAR_DAYS or ACTIVE_DAYS_ONLY
                fill: nonTradingDayFillMethod :  PREVIOUS_VALUE, NIL_VALUE
                Options can be selected these are outlined in “Reference Services and Schemas Guide.”    
            
            Output:
                A list containing as many dataframes as requested fields          
        """

                ############### Create request ###############
    
        request = self.refDataSvc.createRequest('HistoricalDataRequest')
        
        # Put field and securities in list if single value is passed
        if type(strFields) == str:
            strFields = [strFields]
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
    
        # Set paramaters :
        for strF in strFields:
            request.append('fields', strF)
        for strS in strSecurity:
            request.append('securities', strS)
        request.set('startDate', startdate.strftime('%Y%m%d'))
        request.set('endDate', enddate.strftime('%Y%m%d'))
        request.set('periodicitySelection', per)
        request.set('periodicityAdjustment', perAdj)
        request.set('nonTradingDayFillOption', days)
        request.set('nonTradingDayFillMethod', fill)
        request.set('currency', curr)
    
                ############### Send request ###############
                
        requestID = self.session.sendRequest(request)
        print("Sending request")
        
                ############### Receive request ###############
        
        dict_Security_Fields = {}
        dict_tickers = {}
        list_msg = []
        
        for field in strFields:
            globals()['dict_'+ field]= {}
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue      
                        
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__()
            
            # Fill message list
            list_msg.append(msg)
                   
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break        

                ############### Exploit data ###############
        
        for msg in list_msg:
            ticker = msg.getElement(SECURITY_DATA).getElement(SECURITY).getValue()
 
            # Create dict for each field
            for field in strFields:
                globals()['dict_'+ field][ticker] = {}
                
            for field_data in msg.getElement(SECURITY_DATA).getElement(FIELD_DATA):
                dat = field_data.getElement(0).getValue()
                
                for i in range(1, (field_data.numElements())):
                    field_name = str(field_data.getElement(i).name())
                    try:
                        globals()['dict_'+field_name][ticker][dat] = field_data.getElement(i).getValueAsFloat()
                    except:
                        globals()['dict_'+field_name][ticker][dat] = field_data.getElement(i).getValueAsString()
                        
            for field in strFields:
                dict_Security_Fields[field] = pd.DataFrame.from_dict(globals()['dict_'+ field], orient='columns')
                
        return dict_Security_Fields    
    
    #-----------------------------------------------------------------------------------------------------
    #------------------------------------------- BDP -----------------------------------------------------
    #-----------------------------------------------------------------------------------------------------
    
    def bdp(self, strSecurity, strFields, strOverrideField='', strOverrideValue=''):
        
        """
            Summary:
                Reference Data Request ; Real-time if entitled, else delayed values 
                Only supports 1 override
                
            Input:
                strSecurity (list of str) : list of tickers
                strFields (list of str) : list of fields, must be static fields (e.g. px_last instead of last_price)
                strOverrideField
                strOverrideValue         
            
            Output:
               Dict 
        """
        
                ############### Create request ###############

        request = self.refDataSvc.createRequest('ReferenceDataRequest')
        
        # Put field and securities in list is single field passed
        if type(strFields) == str:
            strFields = [strFields]
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
            
        # Append list of fields & securities 
        for strD in strFields:
            request.append('fields', strD)
        for strS in strSecurity:
            request.append('securities', strS)

        # Add override 
        if strOverrideField != '':
            o = request.getElement('overrides').appendElement()
            o.setElement('fieldId', strOverrideField)
            o.setElement('value', strOverrideValue)

                ############### Send request ###############

        requestID = self.session.sendRequest(request)
        print("Sending request")
        
                ############### Receive request ###############             
                
        list_msg = []
        dict_Security_Fields = {}
        dict_tickers = {}
        list_pd = []
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
            
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__()
            
            # Fill message list
            list_msg.append(msg)
            
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break    

                ############### Extract the data ###############

        for msg in list_msg:
            for sec_data in msg.getElement(SECURITY_DATA): # Ticker
                ticker = sec_data.getElement(SECURITY).getValue()
                dict_Security_Fields = {}
                for field in sec_data.getElement(FIELD_DATA) : # Fields
                    dict_Security_Fields[field.name()] = field.getValue()
                dict_tickers[ticker] = pd.DataFrame.from_dict(dict_Security_Fields, orient='index')
                dict_tickers[ticker].columns = [ticker] 
                list_pd.append(dict_tickers[ticker].T)
        
        return pd.concat(list_pd)
    
    #-----------------------------------------------------------------------------------------------------
    #------------------------------------------- BDS -----------------------------------------------------
    #-----------------------------------------------------------------------------------------------------

    def bds(self, strSecurity, strFields, snapshot_date, curr=None):
        
        """
            Summary:
                Reference Data Request; retrieves snapshot data for a set of securities and fields for a specific date.

            Inputs:
                strSecurity: list of str: list of tickers
                strFields: list of str: list of fields, must be static fields (e.g., px_last instead of last_price)
                snapshot_date: date: the specific date for which you want the snapshot data
                curr: string, else default currency is used

            Output:
                A DataFrame containing snapshot data for the requested securities and fields at the specified date
        """
           
                ############### Create request ###############
    
        request = self.refDataSvc.createRequest('ReferenceDataRequest')
        
        # Put field and securities in list if single value is passed
        if type(strFields) == str:
            strFields = [strFields]
        if type(strSecurity) == str:
            strSecurity = [strSecurity]
    
        # Set paramaters :
        for strF in strFields:
            request.append('fields', strF)
        for strS in strSecurity:
            request.append('securities', strS)
        
        for strS in strSecurity:
            overrides = request.getElement('overrides')
            override = overrides.appendElement()
            override.setElement('fieldId', 'REFERENCE_DATE')
            override.setElement('value', snapshot_date.strftime('%Y%m%d'))
        
                ############### Send request ###############

        requestID = self.session.sendRequest(request)
        print("Sending request")
        
                ############### Receive request ###############
        
        list_msg = []
        dict_Security_Fields = {}
        list_compo = []
        
        while True:
            event = self.session.nextEvent()
            
            # Ignores anything that's not partial or final
            if (event.eventType() !=blpapi.event.Event.RESPONSE) & (event.eventType() !=blpapi.event.Event.PARTIAL_RESPONSE):
                continue
            
            # Extract the response message
            msg = blpapi.event.MessageIterator(event).__next__() 
            
            # Fill message list
            list_msg.append(msg)
            
            # Break loop if response is final
            if event.eventType() == blpapi.event.Event.RESPONSE:
                break        
        
                ############### Exploit data ###############
         
        list_compo = []
    
        for msg in list_msg:
            for sec_data in msg.getElement(SECURITY_DATA):  # Ticker
                ticker = sec_data.getElement(SECURITY).getValue()
                dict_Security_Fields = {}
                for member_data in sec_data.getElement(FIELD_DATA).getElement("INDX_MEMBERS"):
                    member = member_data.getElementAsString("Member Ticker and Exchange Code") 
                    list_compo.append(member)
                dict_Security_Fields[ticker] = pd.DataFrame(list_compo, columns=[snapshot_date])
    
        return dict_Security_Fields
    
    def closeSession(self):
        print("Session closed")
        self.session.stop()

In [4]:
####################################################################################################################
################################################# Data importation #################################################
####################################################################################################################
     
# Cette partie pourras être mis dans le fichier data.py après les tests :)      
     
blp = BLP()

Session open


____

# Récupération des compositions : 

In [5]:
start_date = datetime(1999, 1, 28)
end_date = datetime.now()
dates_list = [start_date + i * timedelta(days=30) for i in range((end_date - start_date).days // 30 + 1)]

In [158]:
strFields = ["INDX_MEMBERS"] 
tickers = ["RIY Index"] # RUSSEL

In [145]:
df_compo = pd.DataFrame()
for date in dates_list :
    results = blp.bds(strSecurity=tickers, strFields = strFields, snapshot_date = date)
    new_column = results[tickers[0]]
    df_compo[date] = new_column[date]

Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending request
Sending 

C:\Users\kircca22\AppData\Local\Temp\ipykernel_23292\3022018798.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_compo[date] = new_column[date]


Sending request


C:\Users\kircca22\AppData\Local\Temp\ipykernel_23292\3022018798.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_compo[date] = new_column[date]


Sending request


...

C:\Users\kircca22\AppData\Local\Temp\ipykernel_23292\3022018798.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_compo[date] = new_column[date]


Sending request


C:\Users\kircca22\AppData\Local\Temp\ipykernel_23292\3022018798.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_compo[date] = new_column[date]


Sending request


C:\Users\kircca22\AppData\Local\Temp\ipykernel_23292\3022018798.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_compo[date] = new_column[date]


In [6]:
file_name = "Bloomberg_data_compo.xlsx"
df_compo = pd.read_excel(file_name)

In [7]:
df_compo = df_compo.drop(df_compo.columns[0], axis=1)
df_compo

,1999-01-28,1999-02-27,1999-03-29,1999-04-28,1999-05-28,1999-06-27,1999-07-27,1999-08-26,1999-09-25,1999-10-25,...,2023-05-22,2023-06-21,2023-07-21,2023-08-20,2023-09-19,2023-10-19,2023-11-18,2023-12-18,2024-01-17,2024-02-16
0,A UN,A UN,A UN,A UN,A UN,A UN,A UN,A UN,A UN,A UN,...,A UN,A UN,A UN,A UN,A UN,A UN,A UN,A UN,A UN,A UN
1,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,...,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN,AA UN
2,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,...,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW,AAL UW
3,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,...,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN,AAP UN
4,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,...,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW,AAPL UW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,...,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW,ZI UW
1003,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,...,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW,ZION UW
1004,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,...,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW,ZM UW
1005,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,...,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW,ZS UW


In [147]:
file_name_compo = "Bloomberg_data_compo.xlsx"
df_compo.to_excel(file_name_compo, index=True)

# Applatissement des tickers : 

In [8]:
flattened_data = df_compo.values.ravel()
# Supprimer les doublons en convertissant la liste en un ensemble
flattened_data_unique = list(set(flattened_data))

In [9]:
print(len(flattened_data_unique))

1007


In [12]:
list_tickers = [ticker + " Equity" for ticker in flattened_data_unique]

In [13]:
list_tickers

['BOKF UW Equity',
 'CCCS UW Equity',
 'LII UN Equity',
 'SUI UN Equity',
 'CBRE UN Equity',
 'AVTR UN Equity',
 'MSI UN Equity',
 'INVH UN Equity',
 'CAG UN Equity',
 'MKC UN Equity',
 'USB UN Equity',
 'ROL UN Equity',
 'DVA UN Equity',
 'DLB UN Equity',
 'MA UN Equity',
 'SHW UN Equity',
 'NVDA UW Equity',
 'VNT UN Equity',
 'DAL UN Equity',
 'PYCR UW Equity',
 'CUBE UN Equity',
 'CTVA UN Equity',
 'VSAT UW Equity',
 'VSCO UN Equity',
 'CB UN Equity',
 'AOS UN Equity',
 'PARAA UW Equity',
 'K UN Equity',
 'AZTA UW Equity',
 'NOW UN Equity',
 'PAYC UN Equity',
 'FTRE UW Equity',
 'LIN UW Equity',
 'CDNS UW Equity',
 'HLI UN Equity',
 'ADT UN Equity',
 'DGX UN Equity',
 'CVX UN Equity',
 'HII UN Equity',
 'MGM UN Equity',
 'AMED UW Equity',
 'LAD UN Equity',
 'REG UW Equity',
 'HRB UN Equity',
 'DG UN Equity',
 'GGG UN Equity',
 'BHF UW Equity',
 'VST UN Equity',
 'IT UN Equity',
 'FSLR UW Equity',
 'BRK/B UN Equity',
 'MTN UN Equity',
 'CVS UN Equity',
 'NI UN Equity',
 'PEP UW Equit

In [ ]:
#### Récupération des données : 

In [14]:
# test bdh :
tickers = list_tickers
strFields = ["PX_LAST", "PX_VOLUME"]  
start_date = datetime(1999, 1, 28)
end_date = datetime.now()
test_bdh = blp.bdh(strSecurity=tickers, strFields = strFields, startdate = start_date, enddate = end_date, per='MONTHLY', curr = "USD")

Sending request


In [15]:
test_bdh["PX_LAST"]

,LIN UW Equity,CDNS UW Equity,HLI UN Equity,ADT UN Equity,DGX UN Equity,CVX UN Equity,HII UN Equity,MGM UN Equity,HWM UN Equity,SIRI UW Equity,...,PCOR UN Equity,KLAC UW Equity,FCN UN Equity,TKO UN Equity,UNH UN Equity,KRC UN Equity,ALLY UN Equity,ETR UN Equity,CTRA UN Equity,MDB UQ Equity
2023-11-30,413.77,273.27,107.72,5.87,137.23,143.60,237.02,39.44,52.60,4.68,...,59.09,544.62,220.46,77.33,552.97,32.98,29.22,101.41,26.2500,415.74
2023-12-29,410.71,272.37,119.91,6.82,137.88,149.16,259.64,44.68,54.12,5.47,...,69.22,581.30,199.15,81.58,526.47,39.84,34.92,101.19,25.5200,408.85
2024-01-31,404.83,288.46,119.78,6.53,128.43,147.43,258.92,43.37,56.26,5.09,...,71.39,594.04,191.61,83.69,511.74,35.76,36.68,99.76,24.8800,400.52
2024-02-29,448.82,304.38,128.66,7.26,124.89,152.01,291.62,43.28,66.55,4.42,...,78.03,682.30,206.88,83.73,493.60,37.89,36.99,101.57,25.7800,447.58
2005-10-31,NaN,15.98,NaN,NaN,46.71,57.07,NaN,37.37,NaN,6.24,...,NaN,46.29,27.37,12.75,57.89,56.15,NaN,70.72,5.7238,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-05-31,NaN,NaN,NaN,NaN,52.50,53.78,NaN,38.09,NaN,6.04,...,NaN,45.41,21.99,10.84,48.58,45.18,NaN,71.83,3.9188,NaN
2005-06-30,NaN,NaN,NaN,NaN,53.27,55.92,NaN,39.58,NaN,6.48,...,NaN,43.70,20.90,11.42,52.14,47.49,NaN,75.55,4.3375,NaN
2005-07-29,NaN,NaN,NaN,NaN,51.34,58.01,NaN,45.45,NaN,6.82,...,NaN,51.70,24.10,12.29,52.30,52.10,NaN,77.94,5.0650,NaN
2005-08-31,NaN,NaN,NaN,NaN,49.98,61.40,NaN,42.26,NaN,6.88,...,NaN,50.76,24.90,12.63,51.50,52.71,NaN,74.91,5.3988,NaN


In [16]:
test_bdh["PX_VOLUME"]

,LIN UW Equity,CDNS UW Equity,HLI UN Equity,ADT UN Equity,DGX UN Equity,CVX UN Equity,HII UN Equity,MGM UN Equity,HWM UN Equity,SIRI UW Equity,...,PCOR UN Equity,KLAC UW Equity,FCN UN Equity,TKO UN Equity,UNH UN Equity,KRC UN Equity,ALLY UN Equity,ETR UN Equity,CTRA UN Equity,MDB UQ Equity
2023-11-30,10634426.0,11444536.0,1740233.0,15572365.0,6247773.0,50618304.0,2235657.0,30280213.0,15267236.0,69760439.0,...,8535250.0,9575219.0,1368595.0,8769793.0,17780663.0,6320300.0,23338592.0,11150782.0,36876840.0,7973622.0
2023-12-29,16291406.0,13025344.0,7450004.0,18998930.0,5853388.0,55108197.0,2168774.0,29914106.0,15916834.0,86206934.0,...,8014701.0,7619361.0,1744691.0,7725115.0,20911035.0,8825512.0,29642560.0,10311329.0,41475873.0,13485629.0
2024-01-31,12369008.0,12909159.0,2550704.0,16791385.0,7629315.0,57023477.0,1916737.0,26329179.0,18128018.0,74957179.0,...,5084480.0,9409124.0,1175350.0,8399896.0,23717951.0,7965310.0,29736046.0,8701065.0,36930641.0,7904699.0
2024-02-29,14602979.0,11828671.0,2143257.0,20260496.0,6931785.0,49586508.0,1792371.0,28175327.0,19529964.0,78360737.0,...,7422960.0,7786392.0,1210023.0,6757898.0,16546667.0,7638161.0,23433096.0,9461573.0,35302275.0,7484831.0
2005-10-31,NaN,4559107.0,NaN,NaN,21803400.0,183158300.0,NaN,42267200.0,NaN,419213747.0,...,NaN,42758544.0,7032700.0,1381700.0,88382400.0,3707900.0,NaN,23112200.0,101602400.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-05-31,NaN,NaN,NaN,NaN,16992600.0,130787300.0,NaN,40378800.0,NaN,446969668.0,...,NaN,51787096.0,6858900.0,1525500.0,104344300.0,2703000.0,NaN,18491600.0,80196000.0,NaN
2005-06-30,NaN,NaN,NaN,NaN,14090100.0,128967200.0,NaN,38724900.0,NaN,448234563.0,...,NaN,47522123.0,6007400.0,1700100.0,80228700.0,3469600.0,NaN,15925700.0,65241600.0,NaN
2005-07-29,NaN,NaN,NaN,NaN,11837100.0,103155900.0,NaN,25194700.0,NaN,363596115.0,...,NaN,54084967.0,6233800.0,1546100.0,113367900.0,3222300.0,NaN,15903700.0,46893600.0,NaN
2005-08-31,NaN,NaN,NaN,NaN,23473500.0,158095100.0,NaN,32747800.0,NaN,310768509.0,...,NaN,41083096.0,7179900.0,1093000.0,65790100.0,3414900.0,NaN,20152100.0,53340800.0,NaN


In [17]:
file_name_px_last = "Bloomberg_data_pxlast.xlsx"
test_bdh["PX_LAST"].to_excel(file_name_px_last, index=True)

In [18]:
file_name_px_volume = "Bloomberg_data_pxvolume.xlsx"
test_bdh["PX_VOLUME"].to_excel(file_name_px_volume, index=True)

#### Récupération des compo (bds): 

In [140]:
strFields = ["INDX_MEMBERS"] 
tickers = ["RIY Index"] # RUSSEL 
date = datetime(2024,2,28) # datetime(1999,1,28)
test_bds = blp.bds(strSecurity = tickers, strFields = strFields, snapshot_date = date) #, curr = "USD" 

Sending request


In [141]:
test_bds["RIY Index"]

,2024-02-28
0,A UN
1,AA UN
2,AAL UW
3,AAP UN
4,AAPL UW
...,...
1002,ZI UW
1003,ZION UW
1004,ZM UW
1005,ZS UW


_____

### Récupération des fields (bdh): 

In [46]:
# test bdh :
tickers = ["RIY Index", "SPX Index"]  
strFields = ["PX_LAST", "PX_VOLUME"]  
start_date = datetime(2023, 1, 15)
end_date = datetime.now()
test_bdh = blp.bdh(strSecurity=tickers, strFields = strFields, startdate = start_date, enddate = end_date, per='MONTHLY', curr = "USD")

Sending request


In [47]:
test_bdh

{'PX_LAST':             RIY Index  SPX Index
 2023-01-31   2244.911    4076.60
 2023-02-28   2187.749    3970.15
 2023-03-31   2253.360    4109.31
 2023-04-28   2279.165    4169.48
 2023-05-31   2285.653    4179.83
 2023-06-30   2436.931    4450.38
 2023-07-31   2518.419    4588.96
 2023-08-31   2470.022    4507.66
 2023-09-29   2351.354    4288.05
 2023-10-31   2292.376    4193.80
 2023-11-30   2501.694    4567.80
 2023-12-29   2622.137    4769.83
 2024-01-31   2656.435    4845.65
 2024-02-29   2795.562    5096.27,
 'PX_VOLUME':                RIY Index     SPX Index
 2023-01-31  1.892650e+10  1.308153e+10
 2023-02-28  1.900401e+10  1.268223e+10
 2023-03-31  2.733212e+10  1.852806e+10
 2023-04-28  1.707781e+10  1.171662e+10
 2023-05-31  2.283254e+10  1.501074e+10
 2023-06-30  2.413140e+10  1.560186e+10
 2023-07-31  1.923517e+10  1.291653e+10
 2023-08-31  2.022766e+10  1.312539e+10
 2023-09-29  2.034327e+10  1.350291e+10
 2023-10-31  2.126546e+10  1.449556e+10
 2023-11-30  2.103750e+10

In [48]:
test_bdh["PX_LAST"]

,RIY Index,SPX Index
2023-01-31,2244.911,4076.60
2023-02-28,2187.749,3970.15
2023-03-31,2253.360,4109.31
2023-04-28,2279.165,4169.48
2023-05-31,2285.653,4179.83
2023-06-30,2436.931,4450.38
2023-07-31,2518.419,4588.96
2023-08-31,2470.022,4507.66
2023-09-29,2351.354,4288.05
2023-10-31,2292.376,4193.80


In [52]:
test_bdh["PX_VOLUME"]

,RIY Index,SPX Index
2023-01-31,1.892650e+10,1.308153e+10
2023-02-28,1.900401e+10,1.268223e+10
2023-03-31,2.733212e+10,1.852806e+10
2023-04-28,1.707781e+10,1.171662e+10
2023-05-31,2.283254e+10,1.501074e+10
2023-06-30,2.413140e+10,1.560186e+10
2023-07-31,1.923517e+10,1.291653e+10
2023-08-31,2.022766e+10,1.312539e+10
2023-09-29,2.034327e+10,1.350291e+10
2023-10-31,2.126546e+10,1.449556e+10


____

#### Test bdp :

In [50]:
strFields = ["PX_LAST", "PX_VOLUME"]
tickers = ["RIY Index", "SPX Index"]
date = '20240301'
test_bdp = blp.bdp(strSecurity=tickers, strFields = strFields, strOverrideField = "AMOUNT_OUTSTANDING_AS_OF_DT", strOverrideValue = date )

Sending request


In [51]:
test_bdp

,PX_LAST,PX_VOLUME
RIY Index,2804.119,3.736162e+09
SPX Index,5117.090,2.699788e+09


In [155]:
blp.closeSession()

Session closed
